# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.6.2','generated_entries'
)


"""original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
augmented_data = pd.read_csv(os.path.join(DATA_PATH, 'generated_text.csv'))"""


tot_df = pd.read_csv(os.path.join(DATA_PATH, 'total_df.csv'))

In [7]:
test_data = pd.read_excel(os.path.join('..', '..', '..', '..', 'feedback_output.xlsx'))
test_data = test_data[['Entry']].rename(columns={'Entry':'excerpt'})
test_data = test_data[test_data.excerpt.apply(lambda x: 'NONE' != x.upper())]

In [8]:
"""augmented_data = pd.merge(
    right=original_df.drop(columns=['excerpt']),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)"""

"augmented_data = pd.merge(\n    right=original_df.drop(columns=['excerpt']),\n    left=augmented_data[['entry_id', 'excerpt']],\n    on='entry_id',\n    how='right'\n)"

In [9]:
"""tot_df = pd.concat([original_df, augmented_data])"""

'tot_df = pd.concat([original_df, augmented_data])'

In [10]:
tot_df.shape

(50000, 7)

In [11]:
tot_df = tot_df[
    ['entry_id', 'excerpt', 'lead_id',
     'sectors',
     #'severity',
     #'demographic_groups',
     #'subpillars_1d', 
     #'specific_needs_groups',
     'subpillars_2d', 
     #'affected_groups',
     'pillars_2d',
     #'pillars_1d'
     
    ]
]

## Sagemaker Prep

### Session

Configure SageMaker

In [12]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [13]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [14]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=50_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_data.to_pickle(val_path, protocol=4)

### Estimator Definition

In [15]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [16]:
from sagemaker.pytorch import PyTorch

proportions_negative_examples_test = {
    'sectors':0.16,
    'subpillars_2d':0.3,
    'pillars_2d':0.3,
    'subpillars_1d': 0.69,
    'pillars_1d':0.69,
    'demographic_groups': 0.74,
    'specific_needs_groups': 0.86,
    'affected_groups': 0.35
}
factor_prop_tot_train = 0.05
proportions_negative_examples_train = {
    key:value*factor_prop_tot_train for key, value in proportions_negative_examples_test.items()
}
instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-trials",
    'max_len': 256,
    'epochs': 1,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 256,
    'learning_rate': 5e-5,
    #'training_names':'sectors,subpillars_2d,subpillars_1d,specific_needs_groups,affected_groups,demographic_groups',
    'training_names':'sectors,pillars_2d,subpillars_2d',
    #'training_names':'subpillars_1d,specific_needs_groups,demographic_groups',
    #'training_names':'sectors,subpillars_2d,subpillars_1d,severity,specific_needs_groups,affected_groups,demographic_groups',
    #'train_with_all_positive_examples':True,
    "model_mode":"train",
    "proportions_negative_examples_test": str(proportions_negative_examples_test),
    "proportions_negative_examples_train": str(proportions_negative_examples_train),
    "instance_type": instance_type,
    
    #"numbers_augmentation":"with"
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
    #distribution={"smdistributed": {"dataparallel": {"enabled": False}}}
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [17]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [18]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-10-25 14:14:55 Starting - Starting the training job...
2021-10-25 14:15:35 Starting - Launching requested ML instancesProfilerReport-1635171269: InProgress
...
2021-10-25 14:16:17 Starting - Preparing the instances for training.........
2021-10-25 14:17:58 Downloading - Downloading input data
2021-10-25 14:17:58 Training - Downloading the training image...................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-25 14:21:40,268 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-25 14:21:40,291 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-25 14:21:46,520 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-25 14:21:46,958 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Preparing m

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392140 sha256=eb9ae00a2d19920d094c5fac8a14f3c40a5d8a9f6e0bb22359c1fde65c3dcd95
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591916 sha256=cbf1fd936c31396acb7f2deba36c54d9df9f478ff2e6ad79a34bb93146741545
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.4.2-py3-none-any.whl size=49910 sha256=e23e504ab14bbbcbd0e98a346e9c0f7bb9839b3eda408b9777773c32f9bd312a
  Stored in directory: /root/.cache/pip/wheels/28/61/17/b5ccee30498ab9e21ed6ed0f65d71f79cd252017bc95b6fc00
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=3d833a71eb6835d

      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.21.63
    Uninstalling botocore-1.21.63:
      Successfully uninstalled botocore-1.21.63
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.61.2
    Uninstalling tqdm-4.61.2:
      Successfully uninstalled tqdm-4.61.2
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.0
    Uninstalling s3transfer-0.5.0:
      Successfully uninstalled s3transfer-0.5.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.10.1
    Uninstalling fsspec-2021.10.1:
      Successfully uninstalled fsspec-2021.10.1
  A

[2021-10-25 14:23:30.041 algo-1:85 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-25 14:23:30.231 algo-1:85 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-10-25 14:23:30.232 algo-1:85 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-10-25 14:23:30.232 algo-1:85 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-10-25 14:23:30.233 algo-1:85 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-10-25 14:23:30.233 algo-1:85 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-10-25 14:23:30.326 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-10-25 14:23:30.326 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.position_embeddings.weight count_params:131072
[2021-10-25 14:23:30.327 algo-1:85 INFO hook.

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 99it [00:00, ?it/s]#015Training:   0%|          | 0/800 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/800 [00:00<?, ?it/s] #015Epoch 0:   4%|▍         | 30/800 [00:01<00:48, 15.75it/s]#015Epoch 0:   4%|▍         | 30/800 [00:01<00:48, 15.74it/s, loss=2.26, v_num=0, val_f1_epoch=0.0722, val_loss_epoch=0.679, train_f1=0.499]#015Epoch 0:   8%|▊         | 60/800 [00:03<00:46, 16.02it/s, loss=2.26, v_num=0, val_f1_epoch=0.0722, val_loss_epoch=0.679, train_f1=0.499]#015Epoch 0:   8%|▊         | 60/800 [00:03<00:46, 16.02it/s, loss=1.83, v_num=0, val_f1_epoch=0.0722, val_loss_epoch=0.679, train_f1=0.524]#015Epoch 0:  11%|█▏        | 90/800 [00:05<00:43, 16.51it/s, loss=1.83, v_num=0, val_f1_epoch=0.0722, val_loss_epoch=0.679, train_f1=0.524]#015Epoch 0:  11%|█▏        | 90/800 [00:05<00:43, 16.51i

#015                                                           #033[A#015Epoch 0: 100%|██████████| 843/843 [00:58<00:00, 14.29it/s, loss=0.986, v_num=0, val_f1_epoch=0.647, val_loss_epoch=0.326, train_f1=0.702, val_f1_step=0.573, val_loss_step=0.341]#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 99it [00:00, ?it/s]#015Training:   0%|          | 0/831 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/831 [00:00<?, ?it/s] #015Epoch 0:   4%|▎         | 30/831 [00:01<00:48, 16.50it/s]#015Epoch 0:   4%|▎         | 30/831 [00:01<00:48, 16.50it/s, loss=5.85, v_num=0, val_f1_epoch=0.0519, val_loss_epoch=0.697, train_f1=0.514]#015Epoch 0:   7%|▋         | 60/831 [00:03<00:46, 16.62it/s, loss=5.85, v_num=0, val_f1_epoch=0.0519, val_loss_epoch=0.697, train_f1=0.514]#015Epoch 0:   7%|▋         | 60/831 [00:03<00:46, 16.62it/s, loss=3.18, v_num=0, val_f

#015                                                           #033[A#015Epoch 0: 100%|██████████| 831/831 [00:55<00:00, 14.84it/s, loss=2.53, v_num=0, val_f1_epoch=0.486, val_loss_epoch=0.177, train_f1=0.543, val_f1_step=0.486, val_loss_step=0.191]#015Epoch 0: 100%|██████████| 843/843 [03:32<00:00,  3.96it/s, loss=0.986, v_num=0, val_f1_epoch=0.647, val_loss_epoch=0.326, train_f1=0.702, val_f1_step=0.573, val_loss_step=0.341]
#015Epoch 0: 100%|██████████| 800/800 [05:31<00:00,  2.41it/s, loss=1.67, v_num=0, val_f1_epoch=0.488, val_loss_epoch=0.229, train_f1=0.476, val_f1_step=0.479, val_loss_step=0.228]
2021-10-25 14:23:13.394844: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/core/decorators.py:65: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference


2021-10-25 14:30:21 Uploading - Uploading generated training model
2021-10-25 14:31:01 Completed - Training job completed
ProfilerReport-1635171269: IssuesFound
Training seconds: 784
Billable seconds: 784


In [19]:
assert (1==2)


AssertionError: 

## Code used for deploying and testing models:

In [ ]:
import mlflow
import torch

In [ ]:
model = mlflow.pytorch.load_model(
    's3://deep-mlflow-artifact/16/21a5ece6091b4ddf8b223e78159ce1c7/artifacts/pytorch_model_all',
    map_location=torch.device('cpu')   
)

In [ ]:
model.predict(test_data['excerpt'])

In [ ]:
from mlflow import sagemaker

In [ ]:
sagemaker.deploy(
    'testcpu-pytorch-trained-gpu',
    's3://deep-mlflow-artifact/16/21a5ece6091b4ddf8b223e78159ce1c7/artifacts/pytorch_model_all',
    execution_role_arn=SAGEMAKER_ROLE_ARN,
    image_url="961104659532.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:latest",
    region_name="us-east-1",
    instance_type="ml.c4.xlarge",
    synchronous=False,
    archive=True,
)